In [ ]:
# Parameters
CONTEXTLENGTH = 512  # will be overridden by papermill

this notebook loads tirex and makes predictions using it. Input context data is loaded from data/interim/data.csv. results are saved to data/processed/prediction_matrices/50/.

In [1]:
import pandas as pd
import os
from pyprojroot import here
import torch

import logging
import argparse

import sys
sys.path.append(str(here() / 'scripts'))

from utils import (get_data, get_y, convert_to_tensors, get_x_tensors, save, get_n_epochs)
from chronos import BaseChronosPipeline, ChronosBoltPipeline

/home/valtti/code/ed-ml-multivar/env-chronos/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
MAXMODELCONTEXT= 512 # max context length for this model
MAXCONTEXT = 12840 # max context length for this dataset (2017-01-01 - 2018-06-19)
if CONTEXTLENGTH > MAXMODELCONTEXT:
    CONTEXTLENGTH = MAXMODELCONTEXT

if CONTEXTLENGTH > MAXCONTEXT:
    CONTEXTLENGTH = MAXCONTEXT


MODEL = 'chronosboltbase'
TARGET = 'occ'
FEATURESET = 'u'
TESTNAME = f'{TARGET}-{MODEL}-{FEATURESET}-{CONTEXTLENGTH}'
TESTSTART = pd.Timestamp('20180620')
TESTSIZE = 365*24
PREDICTIONLENGTH = 24

In [ ]:
logging.basicConfig(
    format='%(asctime)s - %(message)s', 
    level=logging.INFO,
    filename=here() / f"logs/logger/{TESTNAME}.log",      
    filemode='w'
    )
logging.info('Starting test.')
logging.info(f'MODEL: {MODEL} TARGET: {TARGET} FEATURESET:'
     f'{FEATURESET} CONTEXTLENGTH: {CONTEXTLENGTH}')

In [3]:
data = get_data(TARGET, MODEL)
y = get_y(data, TARGET)
pc, fc = get_x_tensors(data, FEATURESET)

In [4]:
if TESTSTART in y.time_index:
    start_idx = y.time_index.get_loc(TESTSTART)
else:
    start_idx = y.time_index.get_indexer([TESTSTART], method='nearest')[0]
    pred_idx = 0

y = convert_to_tensors(y)

num_of_days = (y.shape[0] - start_idx) // 24
y_pred = torch.zeros(num_of_days, 24)
y_context = torch.zeros(num_of_days, CONTEXTLENGTH)
for i in range(num_of_days):
    y_context[i] = y[start_idx - CONTEXTLENGTH:start_idx]
    start_idx += 24

In [8]:
model = BaseChronosPipeline.from_pretrained(
        f'amazon/{"chronos-bolt-base"}',
        device_map="cpu",
        dtype=torch.bfloat16, # torch_dtype is deprecated
    )

predictions, mean = model.predict_quantiles(y_context,
        prediction_length=24,
        quantile_levels=[0.5]
    )
y_pred = torch.squeeze(mean)

torch.Size([365, 24])

In [10]:
save(model_name=MODEL+"test", featureset_name=FEATURESET, target_name=TARGET,
    context_length=CONTEXTLENGTH, test_start=TESTSTART, y_pred=y_pred)
logging.info('Results persisted')